In [1]:
# !pip install transformers
# !pip install datasets

# HuggingFace
- https://github.com/huggingface/transformers
- NLP분야의 스타트업
- 다양한 트랜스포머 모델(transformer.models)과 학습 스크립트(transformer.Trainer)를 제공하는 모듈
- 개발자가 자연어 처리 애플리케이션과 서비스를 빠르고 효율적으로 구축하고 배포할 수 있도록 함

## Pipeline
- 특정 작업을 수행하거나 특정 목표를 달성하기 위해 선형 또는 순차적 방식으로 연결된 일련의 프로세스
- preprocess -> 모델 -> post process 생성
- 최초 실행시 모델 다운로드
- pipeline(task, model, config, tokenizer, ...)
  - task : 내가 원하는 작업(ex. 'sentiment-analysis', 'zero-shot-classification', ...)
  - model : 사용모델 (Default=task에 적절한 모델 할당)


In [2]:
from transformers import pipeline

## 감정분석

In [3]:
classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [4]:
classifier.model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [5]:
classifier("I've been waiting for a HuggingFace course my whole life.")

[{'label': 'POSITIVE', 'score': 0.9598049521446228}]

## Zero-shot 분류
- Zero-shot-Learning : 모델이 이전에 본 적이 없는 개체나 개념을 인식하도록 훈련되는 기계 학습
- Zero-shot-Classification : 해당 클래스에 대한 명시적인 훈련 없이 이전에 본 적이 없는 클래스 분류

In [6]:
classifier = pipeline("zero-shot-classification")   # Default Model = facebook/bart-large-mnli
classifier(
    "This is a course about the transformers library",           # 문제
    candidate_labels = ["education", "politics", "business"],    # 정답보기
)

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'sequence': 'This is a course about the transformers library',
 'labels': ['education', 'business', 'politics'],
 'scores': [0.9192408919334412, 0.060778193175792694, 0.01998087950050831]}

In [7]:
classifier(
    "This is a course about the transformers library",          # 문제
    candidate_labels = ["course", "library", "game", "This"],    # 정답보기
)

{'sequence': 'This is a course about the transformers library',
 'labels': ['course', 'library', 'This', 'game'],
 'scores': [0.732907235622406,
  0.19588284194469452,
  0.06776876002550125,
  0.003441136097535491]}

## 생성모델
- 문제로 주어진 문장을 기반으로 그 다음 문장을 생성

In [8]:
generator = pipeline("text-generation")
generator("In this course, we will teach you how to ")

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `ma

[{'generated_text': 'In this course, we will teach you how to \xa0create simple, beautiful, dynamic design diagrams, and how to create them with a variety of basic software tools. We will make use of our favorite tools like Sketch, L.A. Sketch'}]

### 세부조정
- num_return_sequences : 문장갯수
- max_length : 문장 길이

In [9]:
generator("In this course, we will teach you how to ", num_return_sequences=5, max_length=20)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to \xa0communicate with fellow listeners.\n2'},
 {'generated_text': 'In this course, we will teach you how to \xa0install the new web application for PHP 5'},
 {'generated_text': 'In this course, we will teach you how to \xa0understand the most basic \xa0of'},
 {'generated_text': 'In this course, we will teach you how to \xa0compete with the enemy. By taking'},
 {'generated_text': 'In this course, we will teach you how to \xa0explicitly use the\xa0cargo'}]

In [10]:
list_ = ["In this course, we will teach you how to ", "This is a course about the transformers library"]

for sentence in list_:
    print(generator(sentence, num_return_sequences=1, max_length=20))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to \xa0help others get involved in social media.'}]
[{'generated_text': 'This is a course about the transformers library.\n\nThis is part of the Meej'}]


### HuggingFace 사이트에 있는 모델사용
- 사용할 Task, Libraries 등 또는 Filter을 설정하고 원하는 모델을 찾기
- 모델 포스팅 글에서 사용법 확인하고 적용하기
![image](https://user-images.githubusercontent.com/118172599/236658858-5cf4ef58-7005-42f8-8a0e-9682d1ff3d3d.png)
![image](https://user-images.githubusercontent.com/118172599/236659078-0704a318-e5eb-4799-ae90-3367ab3f7ae0.png)
![image](https://user-images.githubusercontent.com/118172599/236659099-61f98aea-1231-4b11-8a31-59160d0ae7d4.png)


In [11]:
generator = pipeline("text-generation", model="huggingtweets/dril")
generator("My dream is ", num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'My dream is ive invented. ive invented what is basically the most popular movie ever made and I need over $10,000 to make it go away. Thank you.'},
 {'generated_text': 'My dream is ive been to be the next "Powerball jack"'},
 {'generated_text': 'My dream is ive come to the realization that i have the power of unlimited consciousness. I would get a brain if i could simply convince all the guys in the house where i live to stop smoking pot that i can still tell the difference between a man and a woman and I would become completely Normal'},
 {'generated_text': "My dream is ive gotten over 1000 jobs. That's what my self believes. Ive fucked over 1000 people"},
 {'generated_text': 'My dream is \ue001� That the \ue003� of humanity \ue006nh is To See The \ue006nht That Is As The \ue001� Of My Dream.'}]

- 사이트에서도 실행가능
![image](https://user-images.githubusercontent.com/118172599/236659251-a18e9e83-462c-4deb-ae82-124ca4e85ddf.png)


## Mask Filling
- `<mask>`에 들어갈 단어 맞추기

In [12]:
unmasker = pipeline("fill-mask")
unmasker("This coures will teach you all about <mask> models", top_k=5)

No model was supplied, defaulted to distilroberta-base and revision ec58a5b (https://huggingface.co/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'score': 0.040895454585552216,
  'token': 745,
  'token_str': ' building',
  'sequence': 'This coures will teach you all about building models'},
 {'score': 0.03127061203122139,
  'token': 30412,
  'token_str': ' mathematical',
  'sequence': 'This coures will teach you all about mathematical models'},
 {'score': 0.025371771305799484,
  'token': 774,
  'token_str': ' role',
  'sequence': 'This coures will teach you all about role models'},
 {'score': 0.01844116672873497,
  'token': 265,
  'token_str': ' business',
  'sequence': 'This coures will teach you all about business models'},
 {'score': 0.015211271122097969,
  'token': 3034,
  'token_str': ' computer',
  'sequence': 'This coures will teach you all about computer models'}]

In [13]:
unmasker("This <mask> will teach you all about classification models", top_k=5)

[{'score': 0.3025355935096741,
  'token': 35950,
  'token_str': ' tutorial',
  'sequence': 'This tutorial will teach you all about classification models'},
 {'score': 0.1506633311510086,
  'token': 1566,
  'token_str': ' article',
  'sequence': 'This article will teach you all about classification models'},
 {'score': 0.06644747406244278,
  'token': 4704,
  'token_str': ' guide',
  'sequence': 'This guide will teach you all about classification models'},
 {'score': 0.06263384968042374,
  'token': 768,
  'token_str': ' course',
  'sequence': 'This course will teach you all about classification models'},
 {'score': 0.05955677106976509,
  'token': 1040,
  'token_str': ' book',
  'sequence': 'This book will teach you all about classification models'}]

## 그룹 엔티티
- 학습되지 않은 단어의 클래스 찾기
- Sylvain: Person, Hugging Face: Organization, Brooklyn: Location

In [14]:
ner = pipeline("ner", grouped_entities=True)
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\pipelines\token_classification.py:168: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


[{'entity_group': 'PER',
  'score': 0.9981694,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.9796019,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9932106,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

## QnA
- Query text : 나는 집에 들어갔다. 그런데 배고파서 햄버거를 먹었다
- Question : 누가 햄버거를 먹었나?
- Answer : 나

In [15]:
question_answer = pipeline("question-answering")
question_answer(
    question="what's my name?",
    context="My name is Sylvain and I work at Hugging Face in Brooklyn.",
)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.9988495111465454, 'start': 11, 'end': 18, 'answer': 'Sylvain'}

## summary
- 한계점 : 긴 문장에서 일부분을 추출해서 요약

In [16]:
summarizer = pipeline("summarization")
summarizer(
    """
    National Commercial Bank (NCB), 
    Saudi Arabia’s largest lender by assets, 
    agreed to buy rival Samba Financial Group for $15 billion in the biggest banking takeover this year.
    NCB will pay 28.45 riyals ($7.58) for each Samba share, according to a statement on Sunday, 
    valuing it at about 55.7 billion riyals. NCB will offer 0.739 new shares for each Samba share, 
    at the lower end of the 0.736-0.787 ratio the banks set when they signed an initial framework 
    agreement in June.The offer is a 3.5% premium to Samba’s Oct. 8 closing price of 27.50 riyals and 
    about 24% higher than the level the shares traded at before the talks were made public. Bloomberg 
    News first reported the merger discussions.The new bank will have total assets of more than $220 billion, 
    creating the Gulf region’s third-largest lender. The entity’s $46 billion market capitalization nearly matches 
    that of Qatar National Bank QPSC, which is still the Middle East’s biggest lender with about $268 billion of 
    assets.
    """
)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'summary_text': " Saudi Arabia's largest lender National Commercial Bank agrees to buy rival Samba Financial Group for $15 billion . NCB will pay 28.45 riyals ($7.58) for each Samba share, valuing it at about 55.7 billion . The new bank will have total assets of more than $220 billion, creating the Gulf region’s third-largest lender ."}]